<a href="https://colab.research.google.com/github/hoodini/cohere-colab-101/blob/main/notebooks/guides/command-a-translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Automated translation from one language to another is one of the oldest applications of machine learning. Today's LLMs have proven remarkably effective for these kinds of tasks, and Command A Translate is Cohere’s state of the art entry into the machine translation field. It delivers industry-leading performance on a variety of translation tasks across 23 languages, while offering enterprises full control of their data through private deployment options.

This cookbook will walk you through how to utilize Command A Translate; for more information, you can check out our [dedicated documentation](https://docs.cohere.com/docs/command-a-translate).

## Getting Set up

First, let's install (or upgrade) the Cohere client.

In [2]:
!pip install --upgrade cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.2 MB/s eta 0:00:00


## Translating a Message

Next, we'll set up Command A Translate to complete a standard translation task.

In [3]:
# 1. Set up your Cohere client, translation prompt and maximum words per chunk
import cohere
from google.colab import userdata

co = cohere.ClientV2(userdata.get('COHERE_API_KEY'))
model = "command-a-translate-08-2025"

target_language = "Spanish"
prompt_template = "Translate everything that follows into {target_language}:\n\n"
max_words = 15  # Set your desired maximum number of words per chunk

# 2. Your source text
text = (
    "Enterprises rely on translation for some of their most sensitive and business-critical documents and cannot risk data leakage, compliance violations, or misunderstandings. Mistranslated documents can reduce trust and have strategic implications."
)


# 3. Define the chunk_split function (from earlier in your notebook)
def chunk_split(text, max_words, threshold=0.8):

    words = text.split()  # Turn the text into a list of words
    chunks = []  # Initialize an empty list to store our chunks
    start = 0  # Starting index for slicing the words list

    while start < len(words):
        # Determine the end index for the current chunk
        end = min(start + max_words, len(words))
        chunk_words = words[start:end]
        chunk_text = " ".join(chunk_words)  # Combine words back into a string

        # If we're at the end of the text or the chunk is too short, add it as is
        if end == len(words) or len(chunk_words) < max_words * threshold:
            chunks.append(chunk_text.strip())
            break

        # Try to find a natural breaking point within the chunk
        split_point = None
        for separator in ["\n", ".", ")", " "]:
            idx = chunk_text.rfind(separator)
            if idx != -1 and idx >= len(chunk_text) * threshold:
                split_point = idx + 1  # Position after the separator
                break

        if split_point:
            # If a good split point is found, add the chunk up to that point
            chunks.append(chunk_text[:split_point].strip())
            # Move the start index forward by the number of words consumed
            consumed = len(chunk_text[:split_point].split())
            start += consumed
        else:
            # If no good split point is found, add the entire chunk
            chunks.append(chunk_text.strip())
            start = end  # Move to the next chunk

    return chunks

# 4. Split the text into chunks using chunk_split
chunks = chunk_split(text, max_words=max_words)

# 5. Translate each chunk and collect results
translated_chunks = []
for chunk in chunks:
    prompt = prompt_template.format(target_language=target_language) + chunk
    response = co.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    translated = response.message.content[0].text
    translated_chunks.append(translated)

# 6. Merge the translated chunks back together
translated_text = " ".join(translated_chunks)

# 7. Output the final translation
print(translated_text)


Las empresas dependen de la traducción para algunos de sus documentos más confidenciales y críticos para su actividad, y no puede arriesgarse a que se produzcan fugas de datos, incumplimientos de la normativa o malentendidos. Los documentos mal traducidos pueden reducir la confianza y tienen implicaciones estratégicas.


## Conclusion

To learn more, check out our dedicated [Command A Translate](/docs/command-a-translate) documentation.